In [23]:
from langchain_community.document_loaders import PyPDFLoader# LangChain Community är en verktyg för dokumenthantering och PyPDFLoader används för att läsa och extrahera
#text från PDF-filer så att de kan bearbetas av språkmodeller.
loader = PyPDFLoader("Foretagsinfo_RSA_AB.pdf")#Det här steget förbereder PDF-filen för att laddas in och bearbetas.
Pdf_data = loader.load()  #  läser in hela PDF-filen

Pdf_data#visar data

[Document(metadata={'producer': 'Microsoft® Word for Microsoft 365', 'creator': 'Microsoft® Word for Microsoft 365', 'creationdate': '2025-05-26T12:50:01+02:00', 'author': 'python-docx', 'moddate': '2025-05-26T12:50:01+02:00', 'source': 'Foretagsinfo_RSA_AB.pdf', 'total_pages': 5, 'page': 0, 'page_label': '1'}, page_content='Företagsprofil: Rsa AB  \n1. Företagsprofil \nRsa AB är ett svenskt IT-konsultföretag med över tio års erfarenhet inom \nmjukvaruutveckling, molntjänster och cybersäkerhet. Företaget grundades med visionen att \ntillhandahålla skräddarsydda, effektiva och säkra IT-lösningar för små och medelstora \nföretag runt om i Sverige. \n \nVår mission är att förenkla våra kunders vardag genom digitalisering och smart teknik. Vi \nsträvar efter att leverera högsta kvalitet och bygga långsiktiga relationer baserade på \nförtroende, kompetens och tillgänglighet. \n2. Våra tjänster \nVi erbjuder ett brett utbud av tjänster inom IT, anpassade för olika branscher och behov: \n \n-

In [24]:
len(Pdf_data)#visar lägnden av data

5

In [25]:
from langchain.text_splitter import RecursiveCharacterTextSplitter#TextSplitter används för att dela upp långa texter i mindre bitar
#RecursiveCharacterTextSplitter är ett smart sätt att dela text baserat på längd och struktur

#split data
text_splitter = RecursiveCharacterTextSplitter(chunk_size=1000)#chunk_size=1000 betyder att varje textbit (chunk) kommer att ha upp till 1000 tecken
docs = text_splitter.split_documents(Pdf_data)#Delar upp den tidigare laddade PDF-datan


print("Total number of documents: ",len(docs))#Skriver ut hur många chunks som har skapats från PDF:en

Total number of documents:  9


In [26]:
import os #används för att läsa miljövariabler, hantera filvägar
from dotenv import load_dotenv#används för att läsa in miljövariabler från en .env-fil och
load_dotenv() #läser in miljövariabler från en fil som heter .env
api_key = os.getenv("GOOGLE_API_KEY")#Hämtar värdet på miljövariabeln



In [27]:
from langchain_google_genai import GoogleGenerativeAIEmbeddings#GoogleGenerativeAIEmbeddings används för att skapa vektorrepresentationer av text med hjälp av Google’s generativa AI-modeller, 
#vilket är användbart för sökning eller liknande texter

embeddings = GoogleGenerativeAIEmbeddings(model="models/gemini-embedding-exp-03-07")#Skapar en instans av Google's generativa AI-embeddingsmodell.
vector = embeddings.embed_query("hello, world!")#Den här raden skickar texten "hello, world!" coh omvandlar med vektor till embeddings-modellen
vector[:5]#Visar de första 5 värdena i vektorn

[-0.024917153641581535,
 0.012005362659692764,
 -0.003886754624545574,
 -0.05774897709488869,
 0.0020742062479257584]

In [28]:
from langchain_core.vectorstores import InMemoryVectorStore#GoogleGenerativeAIEmbeddings används för att skapa vektorrepresentationer av text med hjälp av Google’s generativa AI-modeller, 
#vilket är användbart för sökning eller liknande texter och InMemoryVectorStore lagrar dessa vektorer i minnet

vectorstore = InMemoryVectorStore.from_documents(documents=docs, embedding=GoogleGenerativeAIEmbeddings(model="models/embedding-001"))#skapar
#vektor i minnet från chunkade PDF-dokument

retriever = vectorstore.as_retriever(search_type="similarity", search_kwargs={"k": 2})#Omvandlar vektorbutiken till ett söksystem och  gör en semantisk likhetssökning

retrieved_documents = retriever.invoke("Hur kan jag kontakta er support")#Skickar frågan "Hur kan jag kontakta er support" till retrievern.

print(retrieved_documents)#visar dokements kontent

[Document(id='f0b9b888-8b35-484a-a388-e7dbee8a902c', metadata={'producer': 'Microsoft® Word for Microsoft 365', 'creator': 'Microsoft® Word for Microsoft 365', 'creationdate': '2025-05-26T12:50:01+02:00', 'author': 'python-docx', 'moddate': '2025-05-26T12:50:01+02:00', 'source': 'Foretagsinfo_RSA_AB.pdf', 'total_pages': 5, 'page': 2, 'page_label': '3'}, page_content='9. Kontaktinformation \nRsa AB \nOrg.nr: 556123-4567 \nAdress: Exempelvägen 1, 123 45 Stockholm \nTelefon: 010-123 45 67 \nE-post: info@rsa.se \nWebbplats: www.rsa.se \nÖppettider: Måndag–Fredag: 08:00–17:00 \n \n10. Framtidsvision \nVi ser en framtid där digitalisering spelar en ännu större roll för samhällsutvecklingen. Rsa \nAB strävar efter att vara i framkant och erbjuda innovativa lösningar med fokus på \nhållbarhet, säkerhet och användarvänlighet. Vi planerar att expandera både geografiskt och \ntjänstemässigt för att möta nya behov och utmaningar. \n \n1. Vanliga frågor (FAQ) \nHur kan jag kontakta er support? \n– 

In [29]:
len(retrieved_documents)#visar en lista med matchade av retrieved dokuments

2

In [30]:
# skriva ut själva texten i varje matchat dokument:
for doc in retrieved_documents:
    print(doc.page_content)
    print("-" * 50)


9. Kontaktinformation 
Rsa AB 
Org.nr: 556123-4567 
Adress: Exempelvägen 1, 123 45 Stockholm 
Telefon: 010-123 45 67 
E-post: info@rsa.se 
Webbplats: www.rsa.se 
Öppettider: Måndag–Fredag: 08:00–17:00 
 
10. Framtidsvision 
Vi ser en framtid där digitalisering spelar en ännu större roll för samhällsutvecklingen. Rsa 
AB strävar efter att vara i framkant och erbjuda innovativa lösningar med fokus på 
hållbarhet, säkerhet och användarvänlighet. Vi planerar att expandera både geografiskt och 
tjänstemässigt för att möta nya behov och utmaningar. 
 
1. Vanliga frågor (FAQ) 
Hur kan jag kontakta er support? 
– Du kan nå vår support via e-post på support@rsa.se eller ringa 010-123 45 67. 
2. Har ni jourservice? 
– Ja, vi erbjuder jourservice kvällar och helger för företagskunder med serviceavtal. 
3. Arbetar ni med specifika branscher? 
– Vi är specialiserade på lösningar för hälso- och sjukvård, detaljhandel och 
finanssektorn. 
4. Erbjuder ni skräddarsydda lösningar?
----------------------

In [31]:
from langchain_google_genai import ChatGoogleGenerativeAI#Används för att integrera med Google’s chattbaserade AI-modeller
#används för att anropa en Google Gemini-modell i chatformat
llm = ChatGoogleGenerativeAI(
    model="gemini-2.0-flash",
    temperature=0,
    max_tokens=None,
    
)

In [32]:
from langchain.chains import create_retrieval_chain#Den använder embeddings för att hämta relevanta dokument och sedan ställa frågor mot dem
from langchain.chains.combine_documents import create_stuff_documents_chain#Används för att kombinera flera dokument till ett enda inputblock som matas till språkmodellen
from langchain_core.prompts import ChatPromptTemplate#Används för att definiera mallar och promptar  till AI-modellen ska utformas.



# System prompt och detta är ett meddelande till systemet
system_prompt = (
"Du är en assistent för att besvara frågor."
"Använd följande delar av hämtad kontext för att besvara frågan."
"Om du inte vet svaret, säg att du inte vet."
"Använd högst tre meningar och håll svaret kortfattat.""\n\n{context}"
)
#Skapar en promptmall
prompt = ChatPromptTemplate.from_messages([
    ("system", system_prompt),
    ("human", "{input}"),
])



In [33]:
question_answer_chain = create_stuff_documents_chain(llm, prompt)#Skapar en dokumentkombinerande kedja där alla hämtade dokument
rag_chain = create_retrieval_chain(retriever, question_answer_chain)#Skapar en RAG-kedja

In [34]:
response = rag_chain.invoke({"input": "Hur kan jag kontakta er support?"})#ställa frågor till dina PDF-dokument,
print(response["answer"])#visar svaret

Du kan nå supporten via e-post på support@rsa.se eller ringa 010-123 45 67.
